In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
from math import sqrt
import joblib

In [2]:
# Load the dataset 
data1 = pd.read_excel('Data.xlsx')
data2 = data1.copy(deep=True)

# Check the first few rows of the dataset
print(data1)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

               InvoiceDate  UnitPrice  CustomerID         Country  
0      

In [3]:
print(data1.dtypes)
print('----')
print(data1.isna().sum())
print('----')
print(data1.describe())
print('----')
print(data1[['StockCode', 'Country', 'Description', 'InvoiceNo']].describe())

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object
----
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
----
            Quantity                    InvoiceDate      UnitPrice  \
count  541909.000000                         541909  541909.000000   
mean        9.552250  2011-07-04 13:34:57.156386048       4.611114   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   389

In [4]:
def parse_dtypes(x):
    x['StockCode'] = x['StockCode'].astype(str)
    x['InvoiceNo'] = x['InvoiceNo'].astype(str)
    return x
data1 = parse_dtypes(data1)
print(data1.dtypes)

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object


In [5]:
def filter_data(x):
    # # Filter the DataFrame
    # filtered_df = data1[~data1['InvoiceNo'].str.startswith('c', 'C')]
    # filtered_df
    # Create mask to filter out records with InvoiceNo starting with c
    mask = x['InvoiceNo'].str.startswith(('c', 'C'))
    print(mask.value_counts())
    print('-------------')
    mask=mask.replace(np.nan, False, regex=True)
    print(mask)
    print('--------------')
    y = x[~mask]
    print(y.shape)
    print('--------------')
    # Keep rows with quantity greater than zero
    mask = y['Quantity'] >0
    y =y[mask]
    print(y.shape)
    return y
filtered_data = filter_data(data1)

InvoiceNo
False    532621
True       9288
Name: count, dtype: int64
-------------
0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Name: InvoiceNo, Length: 541909, dtype: bool
--------------
(532621, 8)
--------------
(531285, 8)


In [6]:
def groupby_singledate(x):
    agg_df = x.copy()
    # agg_df_test = filtered_data.groupby([ filtered_data['InvoiceDate'].dt.date, 'StockCode'])['Quantity'].sum()
    # print(agg_df_test)
    agg_df = x.groupby(['StockCode', x['InvoiceDate'].dt.date])['Quantity'].sum().reset_index()
    
    # Function to apply differencing for each product code
    def difference_by_product(df):
        df['Quantity_diff'] = df['Quantity'].diff()
        return df

    # Sort the DataFrame by 'ProductCode' and 'Date'
    agg_df.sort_values(by=['StockCode', 'InvoiceDate'], inplace=True)
    
    # Apply differencing separately for each product code
    agg_df = agg_df.groupby('StockCode').apply(difference_by_product)

    # Drop NaN values created by differencing
    agg_df.dropna(inplace=True)
    agg_df.reset_index(inplace=True, drop=True)
    return  agg_df
agg_df = groupby_singledate(filtered_data)
print(agg_df)

           StockCode InvoiceDate  Quantity  Quantity_diff
0              10002  2010-12-02         1          -59.0
1              10002  2010-12-03         8            7.0
2              10002  2010-12-05         1           -7.0
3              10002  2010-12-06        25           24.0
4              10002  2010-12-07         8          -17.0
...              ...         ...       ...            ...
276376  gift_0001_40  2011-04-04         1            0.0
276377  gift_0001_40  2011-07-27         1            0.0
276378  gift_0001_50  2011-05-06         1            0.0
276379  gift_0001_50  2011-06-24         1            0.0
276380  gift_0001_50  2011-06-30         1            0.0

[276381 rows x 4 columns]


In [8]:
def convert_dates(x):
    x['InvoiceDate']=pd.to_datetime(x['InvoiceDate'])
    x['Year'] = x['InvoiceDate'].dt.year
    x['Month'] = x['InvoiceDate'].dt.month
    x['Day'] = x['InvoiceDate'].dt.day
    x['Dayofweek']=x['InvoiceDate'].dt.dayofweek
    return x
new_agg_df = convert_dates(agg_df)
print(new_agg_df)


           StockCode InvoiceDate  Quantity  Quantity_diff  Year  Month  Day  \
0              10002  2010-12-02         1          -59.0  2010     12    2   
1              10002  2010-12-03         8            7.0  2010     12    3   
2              10002  2010-12-05         1           -7.0  2010     12    5   
3              10002  2010-12-06        25           24.0  2010     12    6   
4              10002  2010-12-07         8          -17.0  2010     12    7   
...              ...         ...       ...            ...   ...    ...  ...   
276376  gift_0001_40  2011-04-04         1            0.0  2011      4    4   
276377  gift_0001_40  2011-07-27         1            0.0  2011      7   27   
276378  gift_0001_50  2011-05-06         1            0.0  2011      5    6   
276379  gift_0001_50  2011-06-24         1            0.0  2011      6   24   
276380  gift_0001_50  2011-06-30         1            0.0  2011      6   30   

        Dayofweek  
0               3  
1          

In [9]:
def standarize_features(x):
    # Create a copy of the DataFrame to avoid modifying the original
    x_std = x.copy()
    
    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Standardize 'Year', 'Month', and 'Day'
    x_std[['Year_normalized', 'Month_normalized', 'Day_normalized']] = scaler.fit_transform(x[['Year', 'Month', 'Day']])

    return x_std

# Apply the function to your DataFrame
new_agg_df2 = standarize_features(new_agg_df)

# # Standardize 'Quantity' separately
# scaler_quantity = StandardScaler()
# new_agg_df2['Quantity_diff_normalized'] = scaler_quantity.fit_transform(new_agg_df2[['Quantity_diff']])
    
print(new_agg_df2)

           StockCode InvoiceDate  Quantity  Quantity_diff  Year  Month  Day  \
0              10002  2010-12-02         1          -59.0  2010     12    2   
1              10002  2010-12-03         8            7.0  2010     12    3   
2              10002  2010-12-05         1           -7.0  2010     12    5   
3              10002  2010-12-06        25           24.0  2010     12    6   
4              10002  2010-12-07         8          -17.0  2010     12    7   
...              ...         ...       ...            ...   ...    ...  ...   
276376  gift_0001_40  2011-04-04         1            0.0  2011      4    4   
276377  gift_0001_40  2011-07-27         1            0.0  2011      7   27   
276378  gift_0001_50  2011-05-06         1            0.0  2011      5    6   
276379  gift_0001_50  2011-06-24         1            0.0  2011      6   24   
276380  gift_0001_50  2011-06-30         1            0.0  2011      6   30   

        Dayofweek  Year_normalized  Month_normalize

In [10]:
def add_avg(x):
    
    # Function to add daily and monthly avg
    def add_daily_monthly_avg(x):
        x['DailyAvg'] = x.groupby(['Dayofweek'])['Quantity_diff'].transform('mean')
        x['MonthlyAvg'] = x.groupby(['Month'])['Quantity_diff'].transform('mean')
        return x

    # Sort the DataFrame by 'ProductCode' and 'Date'
    x.sort_values(by=['StockCode', 'InvoiceDate'], inplace=True)
    
    # Apply differencing separately for each product code
    x = x.groupby('StockCode').apply(add_daily_monthly_avg)
    x.reset_index(inplace=True, drop=True)
    
    return x

new_agg_df3 = add_avg(new_agg_df2)
print(new_agg_df3)

           StockCode InvoiceDate  Quantity  Quantity_diff  Year  Month  Day  \
0              10002  2010-12-02         1          -59.0  2010     12    2   
1              10002  2010-12-03         8            7.0  2010     12    3   
2              10002  2010-12-05         1           -7.0  2010     12    5   
3              10002  2010-12-06        25           24.0  2010     12    6   
4              10002  2010-12-07         8          -17.0  2010     12    7   
...              ...         ...       ...            ...   ...    ...  ...   
276376  gift_0001_40  2011-04-04         1            0.0  2011      4    4   
276377  gift_0001_40  2011-07-27         1            0.0  2011      7   27   
276378  gift_0001_50  2011-05-06         1            0.0  2011      5    6   
276379  gift_0001_50  2011-06-24         1            0.0  2011      6   24   
276380  gift_0001_50  2011-06-30         1            0.0  2011      6   30   

        Dayofweek  Year_normalized  Month_normalize

In [11]:
# Create lagged features (e.g., lagged quantity for 1 and 7 days)
def create_lagged_features(x):
    df = x.copy()
    # Function to add lag values
    def add_lag_values( df):
        df['Quantity_lag_1'] =  df['Quantity_diff'].shift(1)
        df['Quantity_lag_2'] =  df['Quantity_diff'].shift(2)
        df['Quantity_lag_5'] =  df['Quantity_diff'].shift(5)
        df['Quantity_lag_7'] =  df['Quantity_diff'].shift(7)
        return  df

    # Sort the DataFrame by 'ProductCode' and 'Date'
    df.sort_values(by=['StockCode', 'InvoiceDate'], inplace=True)
    
    # Apply differencing separately for each product code
    df =  df.groupby('StockCode').apply(add_lag_values)
    df.reset_index(inplace=True, drop=True)
    df=  df.fillna(0)
    return df

new_agg_df4=create_lagged_features(new_agg_df3)
print(new_agg_df4)

           StockCode InvoiceDate  Quantity  Quantity_diff  Year  Month  Day  \
0              10002  2010-12-02         1          -59.0  2010     12    2   
1              10002  2010-12-03         8            7.0  2010     12    3   
2              10002  2010-12-05         1           -7.0  2010     12    5   
3              10002  2010-12-06        25           24.0  2010     12    6   
4              10002  2010-12-07         8          -17.0  2010     12    7   
...              ...         ...       ...            ...   ...    ...  ...   
276376  gift_0001_40  2011-04-04         1            0.0  2011      4    4   
276377  gift_0001_40  2011-07-27         1            0.0  2011      7   27   
276378  gift_0001_50  2011-05-06         1            0.0  2011      5    6   
276379  gift_0001_50  2011-06-24         1            0.0  2011      6   24   
276380  gift_0001_50  2011-06-30         1            0.0  2011      6   30   

        Dayofweek  Year_normalized  Month_normalize

In [12]:
# Create rolling features (e.g., for window size 3)
def create_rolling_features(x):
    df = x.copy()
    # Function to add lag values
    def add_rolling_values( df):
        df['RollingMean3'] = df['Quantity_diff'].rolling(window=3).mean()
        df['RollingMean5'] = df['Quantity_diff'].rolling(window=5).mean()
        df['RollingMean7'] = df['Quantity_diff'].rolling(window=7).mean()
        
        # df['RollingStd'] = df['Quantity_diff'].rolling(window=7).std()
        # df['RollingSum'] = df['Quantity_diff'].rolling(window=7).sum()
        
        return  df

    # Sort the DataFrame by 'ProductCode' and 'Date'
    df.sort_values(by=['StockCode', 'InvoiceDate'], inplace=True)
    
    # Apply differencing separately for each product code
    df =  df.groupby('StockCode').apply(add_rolling_values)
    df.reset_index(inplace=True, drop=True)
    df=  df.fillna(0)
    return df

new_agg_df5=create_rolling_features(new_agg_df4)
print(new_agg_df5)

           StockCode InvoiceDate  Quantity  Quantity_diff  Year  Month  Day  \
0              10002  2010-12-02         1          -59.0  2010     12    2   
1              10002  2010-12-03         8            7.0  2010     12    3   
2              10002  2010-12-05         1           -7.0  2010     12    5   
3              10002  2010-12-06        25           24.0  2010     12    6   
4              10002  2010-12-07         8          -17.0  2010     12    7   
...              ...         ...       ...            ...   ...    ...  ...   
276376  gift_0001_40  2011-04-04         1            0.0  2011      4    4   
276377  gift_0001_40  2011-07-27         1            0.0  2011      7   27   
276378  gift_0001_50  2011-05-06         1            0.0  2011      5    6   
276379  gift_0001_50  2011-06-24         1            0.0  2011      6   24   
276380  gift_0001_50  2011-06-30         1            0.0  2011      6   30   

        Dayofweek  Year_normalized  Month_normalize

In [13]:
def one_hot_encoding(x):
    # 'Product/Stock Code' should be categorical,
    encoder = LabelEncoder()
    x['StockCodeEncoded'] = encoder.fit_transform(x['StockCode'])
    x = pd.get_dummies(x, columns=[ 'Month', 'Dayofweek' ],  drop_first=True)
    return x
new_agg_df6 = one_hot_encoding(new_agg_df5)
print(new_agg_df6)

           StockCode InvoiceDate  Quantity  Quantity_diff  Year  Day  \
0              10002  2010-12-02         1          -59.0  2010    2   
1              10002  2010-12-03         8            7.0  2010    3   
2              10002  2010-12-05         1           -7.0  2010    5   
3              10002  2010-12-06        25           24.0  2010    6   
4              10002  2010-12-07         8          -17.0  2010    7   
...              ...         ...       ...            ...   ...  ...   
276376  gift_0001_40  2011-04-04         1            0.0  2011    4   
276377  gift_0001_40  2011-07-27         1            0.0  2011   27   
276378  gift_0001_50  2011-05-06         1            0.0  2011    6   
276379  gift_0001_50  2011-06-24         1            0.0  2011   24   
276380  gift_0001_50  2011-06-30         1            0.0  2011   30   

        Year_normalized  Month_normalized  Day_normalized   DailyAvg  ...  \
0             -3.828126          1.384251       -1.532499 

In [14]:
def get_test_and_train(x):
    # Sort the DataFrame by 'date' to ensure data is in the right order
    df = x.copy(deep=True)
    df.sort_values(by=['InvoiceDate'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Do not keep invoice date as index for now
    # df.set_index('InvoiceDate', inplace=True)

    # Split the data into training and testing sets
    train_size = int(len(df) * 0.8)
    train, test = df[:train_size], df[train_size:]
  
    return  train, test

train, test = get_test_and_train(new_agg_df6)
print(train.head())
print('----')
print(test.head())
print('-------')
print(len(train),len(test))

# X = new_agg_df3[features]
# y = new_agg_df3['Quantity_normalized']
# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

  StockCode InvoiceDate  Quantity  Quantity_diff  Year  Day  Year_normalized  \
0     10002  2010-12-02         1          -59.0  2010    2        -3.828126   
1     22465  2010-12-02        48           36.0  2010    2        -3.828126   
2     22466  2010-12-02        24         -438.0  2010    2        -3.828126   
3     22467  2010-12-02       120          111.0  2010    2        -3.828126   
4     22468  2010-12-02         1           -9.0  2010    2        -3.828126   

   Month_normalized  Day_normalized   DailyAvg  ...  Month_8  Month_9  \
0          1.384251       -1.532499   4.666667  ...    False    False   
1          1.384251       -1.532499   1.120000  ...    False    False   
2          1.384251       -1.532499  -0.960000  ...    False    False   
3          1.384251       -1.532499  18.222222  ...    False    False   
4          1.384251       -1.532499   1.250000  ...    False    False   

   Month_10  Month_11  Month_12  Dayofweek_1  Dayofweek_2  Dayofweek_3  \
0     

In [15]:
# Select features and target variable
features = [each for each in new_agg_df6.columns if each not in ['InvoiceDate', 'StockCode', 'Quantity', 'Quantity_diff', 'Year', 'Month', 'Day', 'Year_normalized', 'Month_normalized', 'Day_normalized']]
exog_variables = features
target_varible = 'Quantity_diff'


# Define the SARIMA-X model
order = (1, 1, 1)  # (p, d, q) order of the ARIMA component
seasonal_order = (1, 1, 1, 7)  # (P, D, Q, s) seasonal order

# train = train.astype('float32')
# test = test.astype('float32')
exog = train[exog_variables]
exog = exog.astype('float32')

# Fit the SARIMA-X model
sarima_model = sm.tsa.SARIMAX(train[target_varible], order=order, seasonal_order=seasonal_order, exog=exog)
# sarima_model = sm.tsa.SARIMAX(train['Quantity'], order=order, seasonal_order=seasonal_order)

sarima_results = sarima_model.fit()
print(sarima_results)


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           31     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.12532D+00    |proj g|=  7.85415D-02

At iterate    5    f=  5.10100D+00    |proj g|=  7.82172D-02

At iterate   10    f=  5.05261D+00    |proj g|=  1.36366D-01

At iterate   15    f=  5.04364D+00    |proj g|=  2.71598D-01

At iterate   20    f=  5.00919D+00    |proj g|=  2.90803D-01

At iterate   25    f=  4.96725D+00    |proj g|=  9.23458D-02

At iterate   30    f=  4.93680D+00    |proj g|=  2.49178D-01
  ys=-2.135E-01  -gs= 2.832E-02 BFGS update SKIPPED

At iterate   35    f=  4.93099D+00    |proj g|=  2.42118D-01

At iterate   40    f=  4.93038D+00    |proj g|=  5.87388D-02

At iterate   45    f=  4.92929D+00    |proj g|=  2.70261D-02



 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   50    f=  4.92920D+00    |proj g|=  9.27730D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   31     50    103      2     1     0   9.277D-02   4.929D+00
  F =   4.9292038851334823     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


/Users/david/Desktop/ndsu/stam/ds-project/venv/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [44]:
print(sarima_results.summary())

                                     SARIMAX Results                                     
Dep. Variable:          Quantity_diff_normalized   No. Observations:                55277
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 7)   Log Likelihood               98361.011
Date:                           Mon, 02 Oct 2023   AIC                        -196658.023
Time:                                   20:19:57   BIC                        -196372.584
Sample:                                        0   HQIC                       -196569.025
                                         - 55277                                         
Covariance Type:                             opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Quantity_diff        0.0094   1.39e-06   6770.342      0.000       0.009       0.009
DailyAvg         -3.269e-

In [17]:
print(test.index[0], test.index[-1], test.index.min(), test.index.max())
print(len(exog_variables))

# Make predictions
exog=test[exog_variables]
print(len(exog), len(exog.columns))
exog = exog.astype('float32')

prediction = sarima_results.predict(start=test.index.min(), end=test.index.max(), exog=exog)
print(prediction)



221104 276380 221104 276380
26
55277 26
221104     29.854271
221105     11.717490
221106    -10.594590
221107    -18.235951
221108     -7.030657
             ...    
276376    -64.250728
276377    -25.521364
276378    -30.800933
276379    -39.947778
276380   -106.395752
Name: predicted_mean, Length: 55277, dtype: float64


In [39]:
# Save Model
with open('refine.pkl', 'wb') as f:
    pickle.dump(sarima_results, f)
joblib.dump(sarima_results, 'refine-compress.pkl', compress=3)

In [21]:
# # Convert predictions back to the original scale
# y_pred_original = scaler_quantity.inverse_transform(prediction.values.reshape(-1, 1))
# print(y_pred_original)
test['PredictedDiff'] = prediction
pred_original = test['Quantity'].iloc[0] + test['PredictedDiff'].cumsum()


# Calculate RMSE
rmse = sqrt(mean_squared_error(test['Quantity'], pred_original))
print("Root Mean Squared Error (RMSE):", rmse)

# Output Result File
test['Quantity Prediction'] = pred_original
y = test[['InvoiceDate', 'StockCode',  'Quantity', 'Quantity Prediction']]
y.to_csv("refine.csv")


Root Mean Squared Error (RMSE): 575979.0304319737
